# 2021: Week 9 - Working with Strings

March 03, 2021

Challenge By: Owen Barnes

We have a guest contributor this week! Owen has just finished up his training at the Data School and is a regular Preppin' Data participator. So here's his challenge:

This challenge will be useful for anyone trying to improve their knowledge of Tableau Prep functions, and their data parsing in general. There is an opportunity to use ReGex in this challenge, but there is a longer workaround with other steps available. There will also be the chance to use LODs in this workflow.

We have been given a set of messy strings, which contain useful information that we need to connect to other datasets to eventually find out how much revenue we have generated by selling different products. This string provides us with information such as the quantity of items sold, the product ID code, the phone number of the buyer, and the area code which will let us find out where they are purchasing from. There will also be some small calculations needed to join certain datasets together!

## Inputs

There are three inputs for this week's challenge.

Customer Information: 

<img src='https://1.bp.blogspot.com/-SMTJ0GxSp1U/YCKpJOLwQwI/AAAAAAAAAuQ/E3YyMQUTQZ4q_4-1o720lJYzdaqSml0CQCLcBGAsYHQ/w400-h205/2021W09.1.png'>

Area Code Lookup:

<img src='https://1.bp.blogspot.com/-iZgM1VfejPg/YCKpJN_JseI/AAAAAAAAAuU/9CXJTVZ1C7YxnLGr33doeab33bVpxM8IgCLcBGAsYHQ/w400-h356/2021W09.2.png'>

Product Lookup:

<img src='https://1.bp.blogspot.com/-Vyn5REjLtjE/YCKpJf1iCbI/AAAAAAAAAuY/UR7V8B_Mz54thsX5yfd08fBqI9rPdWbxQCLcBGAsYHQ/s0/2021W09.3.png'>

## Requirements

- Input the Customer Information file, split the values and reshape the data so there is a separate ID on each row. (help1,help2)
- Each ID field contains the following information we need to extract: (help)
    - The first 6 digits present in each ID is the customers phone number
    - The first 2 digits after the ‘,’ is the last 2 digits of the area code 
    - The letter following this is the first letter of the name of the area that they are calling from
    - The digits after this letter resemble the quantity of products ordered
    - The letters after the ‘-‘ are the product ID codes 
- Rename these fields appropriately, and remove any unwanted columns – leaving only these 5 columns in the workflow. 
- Input the Area Code Lookup Table – find a way to join it to the Customer information file (help)
- We don’t actually sell products in Clevedon, Fakenham, or Stornoway. Exclude these from our dataset (help)
- In some cases, the ID field does not provide accurate enough conditions to know where the customer is from. Exclude any phone numbers where the join has produced duplicated records.
- Remove any unwanted fields created from the join. 
- Join this dataset to our product lookup table. 
- For each area, and product, find the total sales values, rounded to zero decimal places (help)
- Rank how well each product sold in each area. (help)
- For each area, work out the percent of total that each different product contributes to the overall revenue of that Area, rounded to 2 decimal places. (help)
- Output the data (updated 03/03/21 17:45 GMT)

## Output

<img src='https://1.bp.blogspot.com/-dzh6MW8N5SE/YD_NaLgKbPI/AAAAAAAAAw8/ArUJI4cdV4Ia-dkjhDi5wRUBcFEOYFcSgCLcBGAsYHQ/s320/image.png'>

- 5 fields
    - Rank
    - Area
    - Product Name
    - Revenue
    - % of Total – Product
- 144 rows (145 including headers)

In [850]:
import pandas as pd

In [851]:
# Input Data
customer = pd.read_excel('Customer Information.xlsx', sheet_name='Sheet1')


In [852]:
# split the values and reshape the data so there is a separate ID on each row
customer = customer['IDs'].str.split(expand=True)
customer = customer.melt(id_vars=None, value_vars=customer.columns, var_name='pivot_name', value_name='ID')
customer.drop(columns='pivot_name', inplace=True)
customer

,ID
0,"Ju856452,13S24-SPL"
1,"jM391563,00C69-SPL"
2,"rV469041,02L68-HS"
3,"GQ505960,03W64-SBP"
4,"bf677129,05D99-SBP"
...,...
995,"RS199602,58E19-HS"
996,"Po054149,59S97-SPL"
997,"Zx736723,61M66-HS"
998,"AL414564,63A65-HS"


In [853]:
# Each ID field contains the following information we need to extract: (help)
# The first 6 digits present in each ID is the customers phone number
# The first 2 digits after the ‘,’ is the last 2 digits of the area code 
# The letter following this is the first letter of the name of the area that they are calling from
# The digits after this letter resemble the quantity of products ordered
# The letters after the ‘-‘ are the product ID codes 


customer['customers_phone_number'] = customer['ID'].str.extract(r'\D+(\d+)')
customer['area_code'] = customer['ID'].str.extract(r'\D+\d+.(\d\d)')
customer['area_calling_from'] = customer['ID'].str.extract(r'\D+\d+.\d\d(\w)')
customer['quantity_of_product'] = customer['ID'].str.extract(r'\D+\d+.\d\d\w(\d+)')
customer['product_id'] = customer['ID'].str.extract(r'\D+\d+.\d\d\w\d+.(\w+)')

# Rename these fields appropriately, and remove any unwanted columns – leaving only these 5 columns in the workflow. 
customer.drop(columns='ID', inplace=True)
customer

,customers_phone_number,area_code,area_calling_from,quantity_of_product,product_id
0,856452,13,S,24,SPL
1,391563,00,C,69,SPL
2,469041,02,L,68,HS
3,505960,03,W,64,SBP
4,677129,05,D,99,SBP
...,...,...,...,...,...
995,199602,58,E,19,HS
996,054149,59,S,97,SPL
997,736723,61,M,66,HS
998,414564,63,A,65,HS


In [854]:
# Input the Area Code Lookup Table – find a way to join it to the Customer information file

area_code = pd.read_excel('Area Code Lookup.xlsx', sheet_name='Sheet1')
area_code

,Code,Area
0,114,Sheffield
1,115,Nottingham
2,116,Leicester
3,117,Bristol
4,118,Reading
...,...,...
748,2894,Antrim
749,2895,Belfast
750,2896,Belfast
751,2897,Saintfield


In [855]:
area_code['key1'] = area_code['Code'].astype(str)
area_code['key1'] = area_code['key1'].str[-2:]
area_code['key2'] = area_code['Area'].str[0]
area_code

,Code,Area,key1,key2
0,114,Sheffield,14,S
1,115,Nottingham,15,N
2,116,Leicester,16,L
3,117,Bristol,17,B
4,118,Reading,18,R
...,...,...,...,...
748,2894,Antrim,94,A
749,2895,Belfast,95,B
750,2896,Belfast,96,B
751,2897,Saintfield,97,S


In [856]:
df1 = pd.merge(left=customer, right=area_code, left_on=['area_calling_from','area_code'], right_on=['key2','key1'])
df1.drop(columns=['key1', 'key2', 'area_code', 'area_calling_from'], inplace=True)
df1

,customers_phone_number,quantity_of_product,product_id,Code,Area
0,856452,24,SPL,18513,Stornoway
1,992446,61,HS,18513,Stornoway
2,762773,85,HS,18513,Stornoway
3,326381,59,SBP,18513,Stornoway
4,019518,73,SPL,18513,Stornoway
...,...,...,...,...,...
1241,297053,97,HS,1499,Inveraray
1242,411896,46,SBP,1499,Inveraray
1243,685201,28,SBP,1499,Inveraray
1244,458822,29,HS,1499,Inveraray


In [857]:
# We don’t actually sell products in Clevedon, Fakenham, or Stornoway. Exclude these from our dataset
exclude_list = ['Clevedon' , 'Fakenham', 'Stornoway']
df1 = df1[~df1['Area'].isin(exclude_list)]
df1

,customers_phone_number,quantity_of_product,product_id,Code,Area
16,391563,69,SPL,1200,Clitheroe
17,391563,69,SPL,1300,Cerne Abbas
18,749573,91,SPL,1200,Clitheroe
19,749573,91,SPL,1300,Cerne Abbas
20,620721,31,SPL,1200,Clitheroe
...,...,...,...,...,...
1241,297053,97,HS,1499,Inveraray
1242,411896,46,SBP,1499,Inveraray
1243,685201,28,SBP,1499,Inveraray
1244,458822,29,HS,1499,Inveraray


In [858]:
# In some cases, the ID field does not provide accurate enough conditions to know where the customer is from. Exclude any phone numbers where the join has produced duplicated records.
counts = df1['customers_phone_number'].value_counts().to_frame().reset_index()
counts = counts.rename(columns={'customers_phone_number':'count', 'index':'customers_phone_number'})
counts = counts[counts['count'] > 1]
duplicate_list = counts['customers_phone_number'].tolist()

df1 = df1[~df1['customers_phone_number'].isin(duplicate_list)]
df1

,customers_phone_number,quantity_of_product,product_id,Code,Area
48,469041,68,HS,1502,Lowestoft
49,672850,68,SPL,1502,Lowestoft
50,524402,35,SBP,1502,Lowestoft
51,638331,63,SBP,1502,Lowestoft
52,872216,96,SBP,1502,Lowestoft
...,...,...,...,...,...
1241,297053,97,HS,1499,Inveraray
1242,411896,46,SBP,1499,Inveraray
1243,685201,28,SBP,1499,Inveraray
1244,458822,29,HS,1499,Inveraray


In [859]:
# Add product table
product = pd.read_excel('Product Lookup.xlsx', sheet_name='Sheet1')

# Clean product table
product['Price'] = product['Price'].str.replace('£','')
product['Price'] = product['Price'].astype(float)
product

,Product ID,Product Name,Price
0,SBP,"Soap, Bar",4.55
1,SPL,"Soap, Liquid",6.50
2,HS,Hand Sanitiser,2.29


In [860]:
# Join this dataset to our product lookup table. 
df2 = pd.merge(left=df1, right=product, left_on='product_id', right_on='Product ID')
df2.drop(columns='product_id', inplace=True)
df2

,customers_phone_number,quantity_of_product,Code,Area,Product ID,Product Name,Price
0,469041,68,1502,Lowestoft,HS,Hand Sanitiser,2.29
1,589776,40,1502,Lowestoft,HS,Hand Sanitiser,2.29
2,987105,21,1502,Lowestoft,HS,Hand Sanitiser,2.29
3,213002,65,1502,Lowestoft,HS,Hand Sanitiser,2.29
4,080932,7,1903,Worthing,HS,Hand Sanitiser,2.29
...,...,...,...,...,...,...,...
746,492463,70,12297,Millom,SBP,"Soap, Bar",4.55
747,183956,0,1499,Inveraray,SBP,"Soap, Bar",4.55
748,411896,46,1499,Inveraray,SBP,"Soap, Bar",4.55
749,685201,28,1499,Inveraray,SBP,"Soap, Bar",4.55


In [861]:
# For each area, and product, find the total sales values, rounded to zero decimal places
df2['quantity_of_product'] = df2['quantity_of_product'].astype(int)
df2['revenue'] = df2['Price'] * df2['quantity_of_product']
df2 = df2.groupby(['Area', 'Product Name']).agg(revenue=('revenue','sum')).reset_index()

# Rank how well each product sold in each area.
df2['rank'] = df2.groupby('Area')['revenue'].rank(method='dense', ascending=False).astype(int)
df2

,Area,Product Name,revenue,rank
0,Aberdeen,Hand Sanitiser,352.66,3
1,Aberdeen,"Soap, Bar",2507.05,1
2,Aberdeen,"Soap, Liquid",793.00,2
3,Arbroath,Hand Sanitiser,847.30,2
4,Arbroath,"Soap, Bar",687.05,3
...,...,...,...,...
139,Worthing,"Soap, Bar",2020.20,1
140,Worthing,"Soap, Liquid",1131.00,2
141,Wrexham,Hand Sanitiser,545.02,3
142,Wrexham,"Soap, Bar",2129.40,1


In [862]:
# For each area, work out the percent of total that each different product 
# contributes to the overall revenue of that Area, rounded to 2 decimal places.

temp_df = df2.groupby('Area').agg(total_revenue=('revenue','sum')).reset_index()
df2 = pd.merge(left=df2, right=temp_df, on='Area')
df2['percent_of_total_product'] = df2['revenue'] / df2['total_revenue'] *100
df2['revenue'] = df2['revenue'].astype(int)
df2['percent_of_total_product'] = df2['percent_of_total_product'].round(2)
df2

,Area,Product Name,revenue,rank,total_revenue,percent_of_total_product
0,Aberdeen,Hand Sanitiser,352,3,3652.71,9.65
1,Aberdeen,"Soap, Bar",2507,1,3652.71,68.64
2,Aberdeen,"Soap, Liquid",793,2,3652.71,21.71
3,Arbroath,Hand Sanitiser,847,2,3958.85,21.40
4,Arbroath,"Soap, Bar",687,3,3958.85,17.35
...,...,...,...,...,...,...
139,Worthing,"Soap, Bar",2020,1,3485.54,57.96
140,Worthing,"Soap, Liquid",1131,2,3485.54,32.45
141,Wrexham,Hand Sanitiser,545,3,3337.42,16.33
142,Wrexham,"Soap, Bar",2129,1,3337.42,63.80


In [863]:
output = df2[['rank','Area','Product Name','revenue','percent_of_total_product']]
output = output.rename(columns={'rank':'Rank', 'revenue':'Revenue', 'percent_of_total_product': r'% of Total - Product'})
output


,Rank,Area,Product Name,Revenue,% of Total - Product
0,3,Aberdeen,Hand Sanitiser,352,9.65
1,1,Aberdeen,"Soap, Bar",2507,68.64
2,2,Aberdeen,"Soap, Liquid",793,21.71
3,2,Arbroath,Hand Sanitiser,847,21.40
4,3,Arbroath,"Soap, Bar",687,17.35
...,...,...,...,...,...
139,1,Worthing,"Soap, Bar",2020,57.96
140,2,Worthing,"Soap, Liquid",1131,32.45
141,3,Wrexham,Hand Sanitiser,545,16.33
142,1,Wrexham,"Soap, Bar",2129,63.80
